In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from datetime import datetime
import re
import math
from sklearn.metrics import make_scorer
from sklearn import preprocessing
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_predict

Populating the interactive namespace from numpy and matplotlib


In [2]:
d = pd.read_csv('tmp/engineered_features_2.csv')
del d['Date']
del d['DateYear']
del d['DateMonth']
del d['DateDay']
del d['Store']
del d['Id']

In [3]:
s = d.Sales
for c in d.columns:
    if (c.startswith('PromoInterval') or
    c.startswith('Competit') or
    (c.startswith('StoreDay') and c != 'StoreDaySales_50') or
    c.startswith('StoreSal') or
    c.startswith('Promo2Sin')):
        del d[c]

In [4]:
d = d.fillna(0)
e = DataFrame(preprocessing.scale(d), columns=d.columns)
d = e
d.Sales = s

In [5]:
def rmspe_func(y_true, y_pred, **kwargs):
    if len(y_pred) != len(y_true):
        raise Exception("different length series passed to scorer function")
    d = DataFrame({'yt': y_true, 'yp': y_pred})
    d = d[d.yt != 0]
    return np.sqrt((1.0/len(d.yt))*(np.square(((d.yt - d.yp)/d.yt)).sum()))
print rmspe_func(Series([2.0, 2.0, 0.0, 0.0]), Series([1.0, 5.0, 100.0, 0.0]))
print rmspe_func(Series([2.0, 2.0, 0.0, 0.0]), Series([0.0, 0.0, 0.0, 0.0]))
rmspe = make_scorer(rmspe_func, greater_is_better=False)

1.11803398875
1.0


In [6]:
d = d[pd.notnull(d.Sales)]

In [7]:
lim_tr = int(0.4 * len(d))
tr = d[:lim_tr]
trl = tr.Sales
del tr['Sales']
lim_ho = int(0.45 * len(d))
ho = d[lim_tr:lim_ho]
hol = ho.Sales
del ho['Sales']
del d

In [8]:
def eval_pred(p):
    return rmspe_func(hol, p)
print eval_pred(zeros_like(hol))

1.0


In [9]:
def eval_model(m):
    m.fit(tr, trl)
    p = m.predict(ho)
    return eval_pred(p)

In [11]:
# eval_model(KNeighborsRegressor(
#     n_neighbors=35,
#     weights='distance'
#     ))
# # 0.1554

0.15540681335762116

In [13]:
# eval_model(KNeighborsRegressor(
#     n_neighbors=15,
#     weights='distance'
#     ))
# # 0.1557

0.15572942078808572

In [16]:
d = pd.read_csv('tmp/engineered_features_2.csv')
del d['Date']
del d['DateYear']
del d['DateMonth']
del d['DateDay']
del d['Store']
del d['Id']

In [17]:
s = d.Sales

In [18]:
for c in d.columns:
    if (c.startswith('PromoInterval') or
    c.startswith('Competit') or
    (c.startswith('StoreDay') and c != 'StoreDaySales_50') or
    c.startswith('StoreSal') or
    c.startswith('Promo2Sin')):
        del d[c]

In [19]:
d = d.fillna(0)
e = DataFrame(preprocessing.scale(d), columns=d.columns)
d = e
d.Sales = s

In [20]:
tr = d[pd.notnull(d.Sales)]
trl = tr.Sales
del tr['Sales']
te = d[pd.isnull(d.Sales)]
del te['Sales']

In [21]:
m = KNeighborsRegressor(
    n_neighbors=35,
    weights='distance'
    )

In [22]:
m.fit(tr, trl)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_neighbors=35, p=2, weights='distance')

In [23]:
knn_te = m.predict(te)

In [24]:
sub = DataFrame({'Id': np.arange(1, len(knn_te)+1), 'Sales': knn_te})
sub.to_csv('output/s_knn_01.csv', index=False)